In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# *IMPORTING LIBRARIES*

In [ ]:
import os
import numpy as np
import pandas as pd
import datetime
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt, numpy as np
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import AgglomerativeClustering
from matplotlib.colors import ListedColormap
from sklearn import metrics
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")
np.random.seed(42)
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import pairwise_distances

# *LOADING DATA*

In [ ]:
data = pd.read_csv("/kaggle/input/customer-personality-analysis/marketing_campaign.csv", sep="\t")
print("Number of datapoints:", len(data))
data.head()

# *DATA CLEANING*

In [ ]:
data.info()

In [ ]:
#To remove the NA values
data = data.dropna()
print("The total number of data-points after removing the rows with missing values are:", len(data))

In [ ]:
data["Dt_Customer"] = pd.to_datetime(data["Dt_Customer"], dayfirst=True)
dates = []
for i in data["Dt_Customer"]:
    i = i.date()
    dates.append(i)  
#Dates of the newest and oldest recorded customer
print("The newest customer's enrolment date in therecords:",max(dates))
print("The oldest customer's enrolment date in the records:",min(dates))

In [ ]:
# how many days each customer has been a customer 
#relative to the most recent customer's enrollment date.
#Created a feature "Customer_For"
days = []
d1 = max(dates) #taking it to be the newest customer
for i in dates:
    delta = d1 - i
    days.append(delta)
data["Customer_For"] = days
data["Customer_For"] = pd.to_numeric(data["Customer_For"], errors="coerce")

In [ ]:
print("Total categories in the feature Marital_Status:\n", data["Marital_Status"].value_counts(), "\n")
print("Total categories in the feature Education:\n", data["Education"].value_counts())

In [ ]:
#Feature Engineering
#Age of customer today 
data["Age"] = 2025-data["Year_Birth"]

#Total spendings on various items
data["Spent"] = data["MntWines"]+ data["MntFruits"]+ data["MntMeatProducts"]+ data["MntFishProducts"]+ data["MntSweetProducts"]+ data["MntGoldProds"]

#Deriving living situation by marital status"Alone"
data["Living_With"]=data["Marital_Status"].replace({"Married":"Partner", "Together":"Partner", "Absurd":"Alone", "Widow":"Alone", "YOLO":"Alone", "Divorced":"Alone", "Single":"Alone",})

#Feature indicating total children living in the household
data["Children"]=data["Kidhome"]+data["Teenhome"]

#Feature for total members in the householde
data["Family_Size"] = data["Living_With"].replace({"Alone": 1, "Partner":2})+ data["Children"]

#Feature pertaining parenthood
data["Is_Parent"] = np.where(data.Children> 0, 1, 0)

#Segmenting education levels in three groups
data["Education"]=data["Education"].replace({"Basic":"Undergraduate","2n Cycle":"Undergraduate", "Graduation":"Graduate", "Master":"Postgraduate", "PhD":"Postgraduate"})

#For clarity
data=data.rename(columns={"MntWines": "Wines","MntFruits":"Fruits","MntMeatProducts":"Meat","MntFishProducts":"Fish","MntSweetProducts":"Sweets","MntGoldProds":"Gold"})

#Dropping some of the redundant features
to_drop = ["Marital_Status", "Dt_Customer", "Z_CostContact", "Z_Revenue", "Year_Birth", "ID"]
data = data.drop(to_drop, axis=1)

In [ ]:
data.describe()


In [ ]:
#To plot some selected features 
#Setting up colors prefrences
sns.set(rc={"axes.facecolor":"#FFF9ED","figure.facecolor":"#FFF9ED"})
pallet = ["#682F2F", "#9E726F", "#D6B2B1", "#B9C0C9", "#9F8A78", "#F3AB60"]
cmap = colors.ListedColormap(["#682F2F", "#9E726F", "#D6B2B1", "#B9C0C9", "#9F8A78", "#F3AB60"])
#Plotting following features
To_Plot = [ "Income", "Recency", "Customer_For", "Age", "Spent", "Is_Parent"]
print("Reletive Plot Of Some Selected Features: A Data Subset")
plt.figure()
sns.pairplot(data[To_Plot], hue= "Is_Parent",palette= (["#682F2F","#F3AB60"]))
#Taking hue 
plt.show()

In [ ]:
#Dropping the outliers by setting a cap on Age and income. 
data = data[(data["Age"]<90)]
data = data[(data["Income"]<600000)]
print("The total number of data-points after removing the outliers are:", len(data))

In [ ]:
# # Step 0: Store original length
# original_len = len(data)

# # Step 1: Select only numeric columns
# numeric_cols = data.select_dtypes(include=['number']).columns

# # Step 2: Remove outliers using IQR for each numeric column
# for col in numeric_cols:
#     Q1 = data[col].quantile(0.25)
#     Q3 = data[col].quantile(0.75)
#     IQR = Q3 - Q1
#     lower_bound = Q1 - 1.5 * IQR
#     upper_bound = Q3 + 1.5 * IQR
#     data = data[(data[col] >= lower_bound) & (data[col] <= upper_bound)]

# # Step 3: Compare lengths
# new_len = len(data)
# print("Original number of data points:", original_len)
# print("Number of data points after removing outliers:", new_len)
# print("Number of data points removed:", original_len - new_len)


In [ ]:
#correlation matrix
corrmat = data.corr(numeric_only=True)
plt.figure(figsize=(20,20))  
sns.heatmap(corrmat, annot=True, cmap=cmap, center=0)

In [ ]:
# Check for missing values
print(data.isnull().sum())



# *DATA PREPROCESSING*

In [ ]:
#Get list of categorical variables
s = (data.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables in the dataset:", object_cols)

In [ ]:
# from sklearn.preprocessing import StandardScaler
# import pandas as pd

# # Drop irrelevant features first
# cols_del = ['AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
#             'AcceptedCmp2', 'Complain', 'Response']
# ds = data.drop(cols_del, axis=1)

# # One-hot encode categorical columns
# ds = pd.get_dummies(ds, columns=object_cols, drop_first=True)

# # Now scale the numeric features
# scaler = StandardScaler()
# scaled_ds = pd.DataFrame(scaler.fit_transform(ds), columns=ds.columns)

# print("All features are now one-hot encoded and scaled.")




In [ ]:
#Label Encoding the object dtypes.
LE=LabelEncoder()
for i in object_cols:
    data[i]=data[[i]].apply(LE.fit_transform)
    
print("All features are now numerical")

In [ ]:
#Creating a copy of data
ds = data.copy()
# creating a subset of dataframe by dropping the features on deals accepted and promotions
cols_del = ['AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1','AcceptedCmp2', 'Complain', 'Response','Kidhome', 'Teenhome', 'Is_Parent','Living_With','Education']
ds = ds.drop(cols_del, axis=1)
#Scaling
scaler = StandardScaler()
scaler.fit(ds)
scaled_ds = pd.DataFrame(scaler.transform(ds),columns= ds.columns )
print("All features are now scaled")

In [ ]:
# cols_del = ['Kidhome', 'Teenhome', 'Family_Size', 'Is_Parent','Living_With', 'Sweets', 'Fish', 'Education']


In [ ]:
#Scaled data to be used for reducing the dimensionality
print("Dataframe to be used for further modelling:")
scaled_ds.head()

# *DIMENSIONALITY REDUCTION*

In [ ]:
X_scaled = scaled_ds.copy()   # Full scaled features

In [ ]:
#Initiating PCA to reduce dimentions aka features to 3
pca = PCA(n_components=3)
pca.fit(X_scaled)

In [ ]:
PCA_ds = pd.DataFrame(pca.transform(scaled_ds), columns=(["col1","col2", "col3"]))
PCA_ds.describe().T

In [ ]:
# pca = PCA(n_components=3)
# reduced = pca.fit_transform(X_scaled)
# plt.scatter(reduced[:, 0], reduced[:, 1])
# plt.title("PCA Projection of Scaled Data")
# plt.show()


In [ ]:
# from sklearn.manifold import TSNE
# tsne = TSNE(n_components=3, random_state=42)
# tsne_result = tsne.fit_transform(X_scaled)
# plt.scatter(tsne_result[:, 0], tsne_result[:, 1])
# plt.title("t-SNE Projection")
# plt.show()


In [ ]:
#A 3D Projection Of Data In The Reduced Dimension
x =PCA_ds["col1"]
y =PCA_ds["col2"]
z =PCA_ds["col3"]
#To plot
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection="3d")
ax.scatter(x,y,z, c="maroon", marker="o" )
ax.set_title("A 3D Projection Of Data In The Reduced Dimension")
plt.show()

In [ ]:
# # Step 0: Store original length
# original_len = len(data)

# # Step 1: Select only numeric columns
# numeric_cols = data.select_dtypes(include=['number']).columns

# # Step 2: Remove outliers using IQR for each numeric column
# for col in numeric_cols:
#     Q1 = data[col].quantile(0.25)
#     Q3 = data[col].quantile(0.75)
#     IQR = Q3 - Q1
#     lower_bound = Q1 - 1.5 * IQR
#     upper_bound = Q3 + 1.5 * IQR
#     data = data[(data[col] >= lower_bound) & (data[col] <= upper_bound)]

# # Step 3: Compare lengths
# new_len = len(data)
# print("Original number of data points:", original_len)
# print("Number of data points after removing outliers:", new_len)
# print("Number of data points removed:", original_len - new_len)


In [ ]:
# X_scaled = scaled_ds.copy()   # Full scaled features

In [ ]:
# from sklearn.cluster import KMeans  
# from sklearn.metrics import silhouette_samples, silhouette_score  
# import matplotlib.pyplot as plt  
# import numpy as np  

# range_n_clusters = range(2, 10)  
# silhouette_scores = []  

# for n_clusters in range_n_clusters:  
#     kmeans = KMeans(n_clusters=n_clusters, random_state=42)  
#     cluster_labels = kmeans.fit_predict(X_scaled)  
#     silhouette_avg = silhouette_score(X_scaled, cluster_labels)  
#     silhouette_scores.append(silhouette_avg)  
#     print(f"For n_clusters={n_clusters}, Silhouette Score={silhouette_avg:.2f}")  

# # Plot the results  
# plt.plot(range_n_clusters, silhouette_scores, marker='o')  
# plt.xlabel("Number of Clusters")  
# plt.ylabel("Silhouette Score")  
# plt.show()  

# Hybrid GA + KMeans Clustering

In [ ]:
# 1. Import required libraries
import random
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise_distances
from deap import base, creator, tools, algorithms
import matplotlib.pyplot as plt
import numpy as np
import random
from scipy.spatial.distance import cdist
from sklearn.metrics import silhouette_score
from collections import Counter
from scipy.spatial.distance import pdist




In [ ]:
# NUM_CLUSTERS = 5             # Number of clusters you want
# POPULATION_SIZE = 100                     # Number of individuals in population
# NUM_GENERATIONS = 500             # Number of generations
# MUTATION_RATE = 0.01              # Probability of mutation
# TOURNAMENT_SIZE = 5             # For tournament selection
# MIN_SAMPLES_PER_CLUSTER = 5  # Minimum points required per cluster# Configuration parameters                     
# ELITISM_RATE = 0.10                   # Percentage of elites to keep
# TOURNAMENT_SIZE_EARLY = 5             # Tournament size early generations
# TOURNAMENT_SIZE_LATE = 7              # Tournament size late generations
# LOCAL_SEARCH_FREQUENCY = 20           # Run local search every N generations
# DIVERSITY_CHECK_FREQUENCY = 25        # Check diversity every N generations
# NUM_CLUSTERS = 5             # Number of clusters you want
# POPULATION_SIZE = 200                     # Number of individuals in population
# NUM_GENERATIONS = 50           # Number of generations
MUTATION_RATE = 0.02              # Probability of mutation
TOURNAMENT_SIZE = 2             # For tournament selection
MIN_SAMPLES_PER_CLUSTER = 6  # Minimum points required per cluster# Configuration parameters                     
ELITISM_RATE = 0.2                   # Percentage of elites to keep
TOURNAMENT_SIZE_EARLY = 2             # Tournament size early generations
TOURNAMENT_SIZE_LATE = 4              # Tournament size late generations
LOCAL_SEARCH_FREQUENCY = 30           # Run local search every N generations
DIVERSITY_CHECK_FREQUENCY = 10        # Check diversity every N generations
PENALTY_VALUE = 1000  # or any value that you want to use as a penalty
MAX_GENERATIONS = 50
NUM_CLUSTERS = 3
POPULATION_SIZE = 200
NUM_GENERATIONS = 50


In [ ]:
# Fitness cache for memoization
fitness_cache = {}

In [ ]:
# X_scaled = scaled_ds.copy()   # Full scaled features

In [ ]:
# NUM_FEATURES = X_scaled.shape[1]

In [ ]:
# print(NUM_FEATURES)


In [ ]:
X = scaled_ds.values


In [ ]:
print(X)


In [ ]:
#For demonstration, we'll create synthetic data
# X_scaled = StandardScaler().fit_transform(X)
np.random.seed(42)
NUM_FEATURES = X_scaled.shape[1] 
# X = np.concatenate([
#     np.random.normal(0, 1, (100, NUM_FEATURES)),
#     np.random.normal(5, 1, (100, NUM_FEATURES)),
#     np.random.normal(10, 1, (100, NUM_FEATURES))
# ])
X=X_scaled

In [ ]:
print(X)


In [ ]:
print(X_scaled)

In [ ]:
INDIVIDUAL_LENGTH = NUM_CLUSTERS * NUM_FEATURES

In [ ]:

max_generations=50
# def create_individual(generation=0, max_generations=200):
#     """Enhanced initialization combining K-Means++, random, and subspace sampling"""
#     # Method 1: K-Means++ initialization (70% probability in early generations)
#     if generation < max_generations * 0.2 and random.random() < 0.7:  
#         if random.random() < 0.7:  # 70% chance for smart initialization
            
#             # Improved K-Means++ with subspace sampling
#             subspace_size = max(3, NUM_FEATURES // 2)
#             features = random.sample(range(NUM_FEATURES), subspace_size)
#             subspace_X = X[:, features]
            
#             centroids = [subspace_X[random.randint(0, len(X)-1)]]
#             for _ in range(1, NUM_CLUSTERS):
#                 distances = np.array([min(np.linalg.norm(x - c) for c in centroids) for x in subspace_X])
#                 probs = distances / (distances.sum() + 1e-8)
#                 new_idx = np.random.choice(len(X), p=probs)
#                 centroids.append(subspace_X[new_idx])
            
#             # Project back to full space
#             full_centroids = np.zeros((NUM_CLUSTERS, NUM_FEATURES))
#             full_centroids[:, features] = np.array(centroids)
#             return full_centroids.flatten()
#         else:  # 30% random with noise
#             return (X_scaled[np.random.choice(len(X), NUM_CLUSTERS)] + 
#                     np.random.normal(0, 0.5, (NUM_CLUSTERS, NUM_FEATURES))).flatten()
#     else:  # Later generations - more exploitation
#         return (X_scaled[random.sample(range(len(X)), NUM_CLUSTERS)] + 
#                 np.random.normal(0, 0.05, (NUM_CLUSTERS, NUM_FEATURES))).flatten()


def create_individual(generation=0, max_generations=200):
    """Enhanced initialization combining K-Means++ and data-driven sampling"""
    # Method 1: K-Means++ initialization (70% probability in early generations)
    if generation < max_generations * 0.2 and random.random() < 0.7:
        indices = [np.random.choice(len(X))]
        for _ in range(1, NUM_CLUSTERS):
            dists = np.min(cdist(X, X[indices]), axis=1)
            probs = dists / (dists.sum() + 1e-8)  # Add small constant to avoid division by zero
            indices.append(np.random.choice(len(X), p=probs))
        return X[indices].flatten()
    
    # Method 2: Random data points with decreasing noise
    noise_scale = 0.5 if generation < max_generations * 0.5 else 0.1
    return (X[np.random.choice(len(X), NUM_CLUSTERS, replace=False)] + 
            np.random.normal(0, noise_scale, (NUM_CLUSTERS, NUM_FEATURES))).flatten()




In [ ]:
def repair_individual(individual, X):
    """Ensure all clusters have at least MIN_SAMPLES_PER_CLUSTER points"""
    centroids = np.array(individual).reshape(NUM_CLUSTERS, NUM_FEATURES)
    distances = cdist(X, centroids, 'euclidean')
    labels = np.argmin(distances, axis=1)
    cluster_counts = np.bincount(labels, minlength=NUM_CLUSTERS)
    
    # Repair empty/small clusters
    for i in range(NUM_CLUSTERS):
        if cluster_counts[i] < MIN_SAMPLES_PER_CLUSTER:
            # Find the farthest point from all centroids in this cluster's region
            if cluster_counts[i] > 0:  # If cluster has some points
                cluster_points = X[labels == i]
                centroid_dist = cdist(cluster_points, [centroids[i]])
                farthest_idx = np.argmax(centroid_dist)
                centroids[i] = cluster_points[farthest_idx]
            else:  # For completely empty clusters
                centroids[i] = X[np.random.choice(len(X))].copy()
    
    return centroids.flatten()

In [ ]:
def create_population(POPULATION_SIZE, generation=0, max_generations=200):
    """Create initial population with enhanced initialization"""
    # return  np.array([create_individual(generation, max_generations) for _ in range(POPULATION_SIZE)])##i think i have to kep it not comment but try
    return [repair_individual(create_individual(generation, max_generations) ,X) for _ in range(POPULATION_SIZE)]

In [ ]:
# # Fitness cache for memoization
# fitness_cache = {}

In [ ]:
# ##thi comment for me i change here the X to X_scaled
# def evaluate_fitness(individual,generation,population=None, apply_sharing=False):
#     """Enhanced fitness evaluation with multiple criteria"""
#     individual_tuple = tuple(individual)
#     if individual_tuple in fitness_cache:
#         return fitness_cache[individual_tuple]
    
#     # Initialize with penalty value by default
#     PENALTY_VALUE = 1000
#     final_fitness = PENALTY_VALUE  # Default penalty value
    
#     try:
#         centroids = np.array(individual).reshape(NUM_CLUSTERS, NUM_FEATURES)
#         distances = cdist(X, centroids, 'euclidean')
#         labels = np.argmin(distances, axis=1).astype(int) 
        
#         # Check cluster validity
#         cluster_counts = np.bincount(labels, minlength=NUM_CLUSTERS)
#         unique_clusters = np.unique(labels)
        
#         if (len(cluster_counts) < 2 or
#            np.any(cluster_counts < MIN_SAMPLES_PER_CLUSTER)):
#            return PENALTY_VALUE

        
#         # Calculate fitness components
#         sil_score = silhouette_score(X, labels)
        
#         intra_cluster = np.mean([np.mean(distances[labels == i, i]) 
#                                for i in range(NUM_CLUSTERS)if cluster_counts[i] > 0])
        
        
#         # inter_cluster = np.min(pairwise_distances(centroids))
#         inter_cluster = np.min(pdist(centroids))
        
#         balance_penalty = np.std(cluster_counts) / len(X)
        
#         # Calculate base fitness
#         base_fitness = - (0.7 * sil_score + 
#                           0.2 * (1/intra_cluster) + 
#                           0.1 * inter_cluster - 
#                           0.2 * balance_penalty-
#                           0.1 * (NUM_CLUSTERS - len(unique_clusters)))
        
#         # Apply fitness sharing if enabled
#         if apply_sharing and population is not None:
#             population = np.array(population) 
#             # sigma_share = 0.3 * NUM_FEATURES
#             sigma_share = (0.25 + 0.15 * (generation/max_generations)) * NUM_FEATURES  # Starts broad (0.4), narrows to 0.25
#             alpha = 1
#             share_sum = 1.0
            

#             neighbor_sample_size = min(
#                 max(20, int(0.3 * len(population))),  # At least 20, or 30% of population
#                 len(population)
#             )
            
#             for other in random.sample(population, neighbor_sample_size):
#                 distance = np.linalg.norm(np.array(individual) - population_array,
#         axis=1
#     )
#                 if distance < sigma_share:
#                     share_sum += 1 - (distance/sigma_share)**alpha
                    
                    
            
#             final_fitness = base_fitness / share_sum
#         else:
#             final_fitness = base_fitness
            
#     except Exception as e:
#         print(f"Error in fitness evaluation: {str(e)}")
#         final_fitness = PENALTY_VALUE
    
#     fitness_cache[individual_tuple] = final_fitness
#     return final_fitness

In [ ]:
##the code run without error but give low score 0.3
# ##thi comment for me i change here the X to X_scaled
# def evaluate_fitness(individual,generation,population=None, apply_sharing=False):
#     """Enhanced fitness evaluation with multiple criteria"""
#     individual_tuple = tuple(individual)
#     if individual_tuple in fitness_cache:
#         return fitness_cache[individual_tuple]
    
#     # Initialize with penalty value by default
#     PENALTY_VALUE = 1000
#     final_fitness = PENALTY_VALUE  # Default penalty value
    
#     try:
#         centroids = np.array(individual).reshape(NUM_CLUSTERS, NUM_FEATURES)
#         distances = cdist(X, centroids, 'euclidean')
#         labels = np.argmin(distances, axis=1).astype(int) 
        
#         # Check cluster validity
#         cluster_counts = np.bincount(labels, minlength=NUM_CLUSTERS)
#         unique_clusters = np.unique(labels)
        
#         if (len(cluster_counts) < 2 or
#            np.any(cluster_counts < MIN_SAMPLES_PER_CLUSTER)):
#            return PENALTY_VALUE

        
#         # Calculate fitness components
#         sil_score = silhouette_score(X, labels)
        
#         intra_cluster = np.mean([np.mean(distances[labels == i, i]) 
#                                for i in range(NUM_CLUSTERS)if cluster_counts[i] > 0])
        
        
#         # inter_cluster = np.min(pairwise_distances(centroids))
#         inter_cluster = np.min(pdist(centroids))
        
#         balance_penalty = np.std(cluster_counts) / len(X)
        
#         # Calculate base fitness
#         base_fitness = - (0.7 * sil_score + 
#                           0.2 * (1/intra_cluster) + 
#                           0.1 * inter_cluster - 
#                           0.2 * balance_penalty-
#                           0.1 * (NUM_CLUSTERS - len(unique_clusters)))
        
#         # Apply fitness sharing if enabled
#         if apply_sharing and population is not None:
#             population = np.array(population) 
#             # sigma_share = 0.3 * NUM_FEATURES
#             sigma_share = (0.25 + 0.15 * (generation/max_generations)) * NUM_FEATURES  # Starts broad (0.4), narrows to 0.25
#             alpha = 1
#             share_sum = 1.0
            

#             neighbor_sample_size = min(
#                 max(20, int(0.3 * len(population))),  # At least 20, or 30% of population
#                 len(population)
#             )
            
#             for other in random.sample(population, neighbor_sample_size):
#                 distance = np.linalg.norm(np.array(individual) - population_array,
#         axis=1
#     )
#                 if distance < sigma_share:
#                     share_sum += 1 - (distance/sigma_share)**alpha
                    
                    
            
#             final_fitness = base_fitness / share_sum
#         else:
#             final_fitness = base_fitness
            
#     except Exception as e:
#         print(f"Error in fitness evaluation: {str(e)}")
#         final_fitness = PENALTY_VALUE
    
#     fitness_cache[individual_tuple] = final_fitness
#     return final_fitness

In [ ]:
def evaluate_fitness(individual, population=None, apply_sharing=False):
    global fitness_cache
    """Enhanced fitness evaluation with multiple criteria"""
    individual_tuple = tuple(individual)
    if individual_tuple in fitness_cache:
        return fitness_cache[individual_tuple]
    
    PENALTY_VALUE = 1000
    final_fitness = PENALTY_VALUE
    
    try:
        centroids = np.array(individual).reshape(NUM_CLUSTERS, NUM_FEATURES)
        distances = cdist(X, centroids, 'euclidean')
        labels = np.argmin(distances, axis=1).astype(int)
        
        # Cluster validity checks
        cluster_counts = np.bincount(labels, minlength=NUM_CLUSTERS)
        unique_clusters = np.unique(labels)
        
        if (len(unique_clusters) < 2 or 
            np.any(cluster_counts < MIN_SAMPLES_PER_CLUSTER)):
            return PENALTY_VALUE
        
        # Calculate metrics
        sil_score = silhouette_score(X, labels)
        intra_cluster = np.mean([np.mean(distances[labels == i, i]) 
                               for i in range(NUM_CLUSTERS) if cluster_counts[i] > 0])
        inter_cluster = np.min(pdist(centroids))if NUM_CLUSTERS > 1 else 0
        balance_penalty = np.std(cluster_counts) / len(X)
        
        # Optimized weight distribution
        base_fitness = - (0.85 * sil_score + 
                         0.10 * (1/intra_cluster) + 
                         0.05 * inter_cluster - 
                         0.02 * balance_penalty - 
                         0.05 * (NUM_CLUSTERS - len(unique_clusters)))
        
        # Fitness sharing if enabled
        if apply_sharing and population is not None:
            sigma_share = 0.3 * NUM_FEATURES
            alpha = 1
            share_sum = 1.0
            
            for other in random.sample(population, min(20, len(population))):
                distance = np.linalg.norm(np.array(individual) - np.array(other))
                if distance < sigma_share:
                    share_sum += 1 - (distance/sigma_share)**alpha
            
            final_fitness = base_fitness / share_sum
        else:
            final_fitness = base_fitness
            
    except Exception as e:
        print(f"Error in fitness evaluation: {str(e)}")
        final_fitness = PENALTY_VALUE
    
    fitness_cache[individual_tuple] = final_fitness
    return final_fitness

In [ ]:
def tournament_selection(population, fitnesses, generation, max_generations, tournament_size=3, elitism=0.15):
    # Gradually increase tournament size
    min_size = 3
    max_size = 7
    diversity = np.mean(pairwise_distances(population))
    tournament_size = min_size + int((generation/max_generations) * (max_size-min_size))
    
    # Keep some randomness in selection pressure
    if random.random() < 0.2:  # 20% chance to vary
        tournament_size = random.randint(min_size, max_size)
        
    selected = []
    elite_count = int(ELITISM_RATE * len(population))
    
    # Elitism: select top elite_count individuals
    if elite_count > 0:
        elite_indices = np.argpartition(fitnesses, elite_count)[:elite_count]
        elites = [population[i] for i in elite_indices]
        selected.extend(elites)
    
    # Tournament selection for the rest
    for _ in range(len(population) - elite_count):
        participants = random.sample(list(zip(population, fitnesses)), tournament_size)
        winner = min(participants, key=lambda x: x[1])  # Assuming minimization
        selected.append(winner[0])
    
    return selected



In [ ]:
def crossover(parent1, parent2, generation, max_generations):
    """SBX crossover returning both children"""
    crossover_rate = 0.9 - (0.5 * generation/max_generations)
    if random.random() > crossover_rate:
        return parent1.copy(), parent2.copy()
    
    p1 = np.array(parent1).reshape(NUM_CLUSTERS, NUM_FEATURES)
    p2 = np.array(parent2).reshape(NUM_CLUSTERS, NUM_FEATURES)
    
    child1 = np.zeros_like(p1)
    child2 = np.zeros_like(p1)
    eta = random.uniform(2, 10)

    for k in range(NUM_CLUSTERS):
        for d in range(NUM_FEATURES):
            u = random.random()
            beta = (2*u)**(1/(eta+1)) if u <= 0.5 else (1/(2*(1-u)))**(1/(eta+1))
            child1[k,d] = 0.5*((1+beta)*p1[k,d] + (1-beta)*p2[k,d])
            child2[k,d] = 0.5*((1-beta)*p1[k,d] + (1+beta)*p2[k,d])

    child1 = np.clip(child1, X.min(axis=0), X.max(axis=0)).flatten()
    child2 = np.clip(child2, X.min(axis=0), X.max(axis=0)).flatten()
    
    return child1, child2

In [ ]:

def mutate(individual, generation, max_generations):
    
    individual = np.array(individual).reshape(NUM_CLUSTERS, NUM_FEATURES)##aff reshape
    
    # base_rate = 0.2 if generation < max_generations*0.3 else 0.1
    # mutation_rate = base_rate + (0.3 * (1 - generation/max_generations))
    mutation_rate = 0.1 + (0.3 * (1 - (generation / max_generations)))  # Starts at 0.4, decays to 0.1

    # Adaptive mutation parameters
    mutation_strength = 0.5 * (1 - generation/max_generations)  # Decreases over time
    mutation_prob = 0.3 * (1 + generation/max_generations)     # Increases slightly

    # Occasionally do large mutations
    if random.random() < 0.1:  # 10% chance for big mutation
        mutation_rate = min(0.5, mutation_rate * 3)
    
    for i in range(len(individual)):
        if random.random() < mutation_prob:##here was mutation rate:
            step_size = max(0.2 * (1 - generation / max_generations), 1e-6)
            individual[i] += np.random.normal(0, step_size)
    
    # --- Reflection Boundary Handling ---
    min_vals = X.min(axis=0)
    max_vals = X.max(axis=0)
    for i in range(NUM_CLUSTERS):
        for j in range(NUM_FEATURES):
            if individual[i, j] < min_vals[j]:
                individual[i, j] = min_vals[j] + (min_vals[j] - individual[i, j])
            elif individual[i, j] > max_vals[j]:
                individual[i, j] = max_vals[j] - (individual[i, j] - max_vals[j])

    # --- K-Means Local Refinement (1 iteration) ---
    kmeans = KMeans(n_clusters=NUM_CLUSTERS, init=individual, n_init=1, max_iter=1)
    kmeans.fit(X)
    refined_centroids = kmeans.cluster_centers_

    return refined_centroids.flatten()


In [ ]:
def local_search_kmeans(individual, X, n_features, fitnesses, max_iter=10):
    """Refine centroids using KMeans with warm-start from individual."""
    n_features = X.shape[1]
    centroids = individual.reshape(NUM_CLUSTERS, NUM_FEATURES)

    try:
        kmeans = KMeans(
            n_clusters=NUM_CLUSTERS,
            init=centroids,
            n_init=1,
            max_iter=max_iter,
            random_state=42
        )
        kmeans.fit(X)
        refined = kmeans.cluster_centers_.flatten()

        #Track improvement
        before = fitnesses(individual)
        after = fitnesses(refined)
        print(f"[LocalSearch] Improved fitness: {before:.4f} -> {after:.4f}")
        
        return refined
    except Exception as e:
        print(f"[WARN] Local search failed: {e}")
        return individual  # fallback in case KMeans fails



In [ ]:
def run_genetic_algorithm():
    # Initialize population and tracking variables
    # population = create_population(POPULATION_SIZE)
    population = [repair_individual(ind, X) for ind in create_population(POPULATION_SIZE)]  # Apply repair during initialization
    best_fitness_history = []
    avg_fitness_history = []
    valid_solutions_history = []
    diversity_history = []

    

    # Initialize fitnesses as a list of fitness values
    fitnesses = [evaluate_fitness(ind) for ind in population]

    print(f"Population size: {len(population)}, Fitnesses size: {len(fitnesses)}")

    for generation in range(max_generations):
        # Adaptive fitness evaluation
        apply_sharing = (generation < max_generations//2) and (generation % 5 == 0)##generation < NUM_GENERATIONS//2 thw change 
        fitnesses = [evaluate_fitness(ind, 
                      population if apply_sharing else None, 
                      apply_sharing) 
                    for ind in population]
        # fitnesses = [evaluate_fitness(individual) for individual in population]
        
        # Process fitness values
        valid_fitnesses = [f for f in fitnesses if f < PENALTY_VALUE]
        valid_count = len(valid_fitnesses)
        best_fit = min(valid_fitnesses) if valid_fitnesses else PENALTY_VALUE
        avg_fit = np.mean(valid_fitnesses) if valid_fitnesses else PENALTY_VALUE
        
        # Track metrics
        best_fitness_history.append(best_fit)
        avg_fitness_history.append(avg_fit)
        valid_solutions_history.append(valid_count)
        

        
        
        # Calculate diversity (moved before selection)
        diversity_history.append(np.mean(pdist(population)))

        
        print(f"Gen {generation+1}: Valid {valid_count}/{len(population)} | Best {-best_fit:.4f} | Avg {-avg_fit:.4f} ")
        
        # Tournament selection
        selected = []
        for _ in range(POPULATION_SIZE):
            candidates = random.sample(range (POPULATION_SIZE), min(5, (POPULATION_SIZE)//2))
            # Select the individual with the best (smallest) fitness
            best_candidate_idx = min(candidates, key=lambda x: fitnesses[x])
            selected.append(population[best_candidate_idx])
            # selected.append(population[min(candidates, key=lambda x: fitnesses[x])])
        
        # Reproduction
        next_gen = []
        for i in range(0, len(selected) - 1, 2):
            p1, p2 = selected[i], selected[i+1]
            c1, c2 = crossover(p1, p2, generation, MAX_GENERATIONS)
            c1 = mutate(c1, generation, MAX_GENERATIONS)
            c2 = mutate(c2, generation, MAX_GENERATIONS)
            next_gen.extend([
                repair_individual(c1,X),
                repair_individual(c2 ,X)
            ])
        
        population = next_gen[:POPULATION_SIZE]

        
        # Diversity maintenance
        if generation % DIVERSITY_CHECK_FREQUENCY == 0:
            pop_array = np.array(population)
            kmeans = KMeans(n_clusters=min(5, len(population)//2)).fit(pop_array)
            cluster_counts = Counter(kmeans.labels_)
            
            for cluster_idx, count in cluster_counts.items():
                if count > len(population)//3:
                    to_replace = count - len(population)//3
                    cluster_indices = np.where(kmeans.labels_ == cluster_idx)[0]
                    replace_indices = np.random.choice(cluster_indices, to_replace, replace=False)
                    for idx in replace_indices:
                        population[idx] = create_individual(generation, NUM_GENERATIONS)

        # fitnesses = evaluate_fitness
        # Local search intensification
        if generation % LOCAL_SEARCH_FREQUENCY == 0:
            best_idx = np.argmin(fitnesses)##fitnesses
            # population[best_idx] = local_search(population[best_idx])
            population[best_idx] = local_search_kmeans(population[best_idx], X, NUM_CLUSTERS, evaluate_fitness)
            fitnesses[best_idx] = evaluate_fitness(population[best_idx], population, apply_sharing)
            

            # Update fitness values after population modification
            fitnesses = [evaluate_fitness(ind, population) for ind in population]

            
            # Replace worst individuals
            worst_indices = np.argsort(fitnesses)[-len(population)//20:]
            for idx in worst_indices:
                population[idx] = create_individual(generation, NUM_GENERATIONS)
                fitnesses[idx] = evaluate_fitness(population[idx], population, apply_sharing)  # Recalculate fitness for replaced individuals


        # Restart mechanism
        if generation == 50 and len(best_fitness_history) > 50:
            if (best_fitness_history[49] - best_fitness_history[0]) < 0.05:
                print("Restarting population for better exploration")
                elite_count = int(POPULATION_SIZE * 0.1)
                elite_indices = np.argpartition(fitnesses, elite_count)[:elite_count]
                elites = [population[i] for i in elite_indices]
                population = elites + create_population(POPULATION_SIZE - elite_count, generation, NUM_GENERATIONS)
                # Clear fitness cache after population restart
                fitness_cache.clear()

    # Final evaluation
    best_idx = np.argmin([evaluate_fitness(ind) for ind in population])
    best_individual = population[best_idx]
    best_centroids = np.array(best_individual).reshape(NUM_CLUSTERS, NUM_FEATURES)
    best_score = -min([evaluate_fitness(ind) for ind in population])  # Positive score

    # Visualization
    visualize_results(X, best_centroids, best_fitness_history, 
                    avg_fitness_history, valid_solutions_history, 
                    diversity_history, NUM_CLUSTERS, POPULATION_SIZE)
    
    return best_centroids, best_score

def visualize_results(X, centroids, best_fitness, avg_fitness, 
                    valid_solutions, diversity, n_clusters, pop_size):
    """Consolidated visualization function"""
    plt.figure(figsize=(18, 12))
    
    # 1. Optimization Progress
    plt.subplot(2, 2, 1)
    plt.plot([-f for f in best_fitness], 'b-', label='Best Fitness')
    plt.plot([-f for f in avg_fitness], 'g-', alpha=0.5, label='Avg Fitness')
    plt.plot([v/pop_size for v in valid_solutions], 'r--', label='Valid Solutions')
    plt.xlabel('Generation')
    plt.ylabel('Score')
    plt.title('Optimization Progress')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # 2. Diversity Tracking
    plt.subplot(2, 2, 2)
    plt.plot(diversity, 'm-')
    plt.xlabel('Generation')
    plt.ylabel('Population Diversity')
    plt.title('Diversity Over Generations')
    plt.grid(True, alpha=0.3)

    # 3. PCA Projection
    plt.subplot(2, 2, 3)
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X)
    centroids_pca = pca.transform(centroids)
    
    colors = plt.cm.viridis(np.linspace(0, 1, n_clusters))
    labels = np.argmin(cdist(X, centroids), axis=1)
    
    for k in range(n_clusters):
        plt.scatter(X_pca[labels == k, 0], X_pca[labels == k, 1],
                    c=[colors[k]], alpha=0.6, label=f'Cluster {k+1}')

    plt.scatter(centroids_pca[:,0], centroids_pca[:,1],
                c='red', marker='X', s=200, edgecolor='black',
                linewidth=1, label='Centroids')
    plt.title('Final Clusters (PCA Projection)')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # 4. Decision Boundaries
    plt.subplot(2, 2, 4)
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1),
                         np.arange(y_min, y_max, 0.1))
    
    mesh_points = np.c_[xx.ravel(), yy.ravel()]
    if X.shape[1] > 2:
        mesh_points = np.hstack([mesh_points, np.zeros((mesh_points.shape[0], X.shape[1]-2))])
    
    Z = np.argmin(cdist(mesh_points, centroids), axis=1)
    Z = Z.reshape(xx.shape)
    
    plt.contourf(xx, yy, Z, alpha=0.3, cmap='viridis')
    plt.scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis', edgecolor='k')
    plt.scatter(centroids[:, 0], centroids[:, 1], c='red', marker='X', s=200)
    plt.title('Decision Boundaries')
    
    plt.tight_layout()
    plt.show()

# Run the algorithm
best_centroids, best_score = run_genetic_algorithm()
print(f"\nBest clustering score: {best_score:.4f}")
print("Final centroids:\n", best_centroids)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

def evaluate_silhouette(X, labels):
    score = silhouette_score(X, labels)
    print(f"Silhouette Score: {score:.4f}")
    return score



# Fit KMeans to the data
kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=42)
kmeans.fit(X)

# Print Silhouette Score
evaluate_silhouette(X, kmeans.labels_)

In [ ]:
# import numpy as np
# import random
# from sklearn.metrics import silhouette_score, pairwise_distances
# from scipy.spatial.distance import cdist
# from sklearn.cluster import KMeans
# from collections import Counter
# import matplotlib.pyplot as plt

# # Configuration parameters
# NUM_CLUSTERS = 3
# POPULATION_SIZE = 75
# NUM_GENERATIONS = 100
# MIN_SAMPLES_PER_CLUSTER = 5
# ELITISM_RATE = 0.15
# TOURNAMENT_SIZE_EARLY = 3
# TOURNAMENT_SIZE_LATE = 7
# LOCAL_SEARCH_FREQUENCY = 50
# DIVERSITY_CHECK_FREQUENCY = 20

# def tournament_selection(population, fitnesses, generation, max_generations, elitism=0.15):
#     """Tournament selection with adaptive size and elitism"""
#     selected = []
#     elite_count = int(elitism * len(population))
    
#     # Elitism: select top elite_count individuals
#     if elite_count > 0:
#         elite_indices = np.argpartition(fitnesses, elite_count)[:elite_count]
#         elites = [population[i] for i in elite_indices]
#         selected.extend(elites)
    
#     # Adaptive tournament size
#     tournament_size = TOURNAMENT_SIZE_EARLY
#     if generation > max_generations * 0.7:
#         tournament_size = TOURNAMENT_SIZE_LATE
    
#     # Tournament selection for the rest
#     for _ in range(len(population) - elite_count):
#         participants = random.sample(list(zip(population, fitnesses)), tournament_size)
#         winner = min(participants, key=lambda x: x[1])  # Select best fitness
#         selected.append(winner[0])
    
#     return selected

# def crossover(parent1, parent2, generation, max_generations, alpha=0.3):
#     """Adaptive crossover with blending"""
#     # Adaptive crossover rate
#     crossover_rate = 0.9 - (0.5 * generation/max_generations)  # Decreases from 0.9 to 0.4
    
#     if random.random() > crossover_rate:
#         return parent1 if random.random() < 0.5 else parent2
    
#     # Blend crossover
#     child = []
#     for p1, p2 in zip(parent1, parent2):
#         # Blend genes within [min - α·range, max + α·range]
#         min_val = min(p1, p2)
#         max_val = max(p1, p2)
#         range_val = max_val - min_val
#         child.append(np.random.uniform(min_val - alpha * range_val, max_val + alpha * range_val))
    
#     return child

# def mutate(individual, generation, max_generations, scale=0.1):
#     """Adaptive mutation with boundary handling"""
#     individual = np.array(individual)
    
#     # Adaptive mutation rate
#     mutation_rate = 0.1 + (0.3 * (1 - generation/max_generations))  # Decreases from 0.4 to 0.1
    
#     for i in range(len(individual)):
#         if random.random() < mutation_rate:
#             # Adaptive mutation step size
#             # step_size = scale * (1 - generation/max_generations)
#             step_size = max(0.2 * (1 - generation / max_generations), 1e-6)
#             individual[i] += np.random.normal(0, step_size)
    
#     # Boundary check
#     min_vals = np.tile(X.min(axis=0), NUM_CLUSTERS)
#     max_vals = np.tile(X.max(axis=0), NUM_CLUSTERS)
#     individual = np.clip(individual, min_vals, max_vals)
    
#     return individual.tolist()

# def runn_genetic_algorithm(X):
#     """Main GA execution function"""
#     # Initialize population and other variables
#     fitness_cache = {}
#     population = create_population(POPULATION_SIZE)
#     best_fitness_history = []
#     valid_solutions_history = []

#     # Main GA loop
#     for generation in range(NUM_GENERATIONS):
#         # Evaluate fitness
#         fitnesses = [evaluate_fitness(ind) for ind in population]
#         best_fit = min(fitnesses)
#         best_fitness_history.append(best_fit)
        
#         # Monitor valid solutions
#         valid_count = sum(1 for f in fitnesses if f > -0.5)
#         valid_solutions_history.append(valid_count)
#         print(f"Gen {generation+1}: Valid {valid_count}/{len(population)} | Best {-best_fit:.4f}")

#         # Selection
#         selected_population = tournament_selection(
#             population, fitnesses, generation, NUM_GENERATIONS, ELITISM_RATE
#         )

#         # Crossover + Mutation
#         next_generation = []
#         for i in range(0, len(selected_population) - 1, 2):
#             parent1, parent2 = selected_population[i], selected_population[i+1]
#             child1 = crossover(parent1, parent2, generation, NUM_GENERATIONS)
#             child2 = crossover(parent2, parent1, generation, NUM_GENERATIONS)
            
#             # Adaptive mutation scale
#             mutation_scale = 0.05 if generation > NUM_GENERATIONS*0.7 else 0.1
#             next_generation.extend([
#                 mutate(child1, generation, NUM_GENERATIONS, scale=mutation_scale),
#                 mutate(child2, generation, NUM_GENERATIONS, scale=mutation_scale)
#             ])
        
#         if len(selected_population) % 2 != 0:
#             next_generation.append(mutate(selected_population[-1], generation, NUM_GENERATIONS))
        
#         population = next_generation[:POPULATION_SIZE]

#         # Diversity maintenance
#         if valid_count < 0.8 * len(population):
#             invalid_indices = np.argsort(fitnesses)[-len(population)//10:]
#             for idx in invalid_indices:
#                 population[idx] = create_individual(generation, NUM_GENERATIONS)
#             print(f"Injected diversity at gen {generation+1}")
        
#         # Late-stage intensification
#         if generation == int(NUM_GENERATIONS*0.7):
#             print("Activating intense optimization mode")
#             current_tournament_size = 7
#             elite_indices = np.argsort(fitnesses)[:10]
#             for i in elite_indices:
#                 population[i] = mutate(population[i], generation, NUM_GENERATIONS, scale=0.05)
#             elite_indices = np.argsort(fitnesses)[:5]
#             elites = [mutate(population[i], generation, NUM_GENERATIONS, scale=0.02) for i in elite_indices]
#             population[-5:] = elites

#     # Final results with enhanced visualization
#     print("\n=== Optimization Complete ===")
#     plt.figure(figsize=(14, 6))

#     # 1. Fitness Progress Plot (Left)
#     plt.subplot(121)
#     plt.plot([-f for f in best_fitness_history], 'b-', label='Best Silhouette', linewidth=2)
#     plt.plot([v/len(population)*100 for v in valid_solutions_history], 'g--', label='Valid Solutions (%)', alpha=0.7)
#     plt.xlabel('Generation', fontsize=12)
#     plt.ylabel('Score', fontsize=12)
#     plt.title('Optimization Progress', fontsize=14)
#     plt.legend(fontsize=10)
#     plt.grid(True, alpha=0.3)

#     # 2. Cluster Visualization (Right)
#     plt.subplot(122)
#     best_centroids = np.array(population[np.argmin(fitnesses)]).reshape(NUM_CLUSTERS, -1)
#     labels = np.argmin(cdist(X, best_centroids), axis=1)

#     # Use distinct colors and markers
#     colors = plt.cm.viridis(np.linspace(0, 1, NUM_CLUSTERS))
#     for k in range(NUM_CLUSTERS):
#         cluster_points = X[labels == k]
#         plt.scatter(cluster_points[:,0], cluster_points[:,1], 
#                    c=[colors[k]], alpha=0.6, label=f'Cluster {k+1}')
        
#     # Plot centroids with outlines
#     plt.scatter(best_centroids[:,0], best_centroids[:,1],
#                c='red', marker='X', s=200, edgecolor='black',
#                linewidth=2, label='Centroids')

#     plt.title(f'Final Clusters (Silhouette: {-best_fit:.4f})', fontsize=14)
#     plt.xlabel('Feature 1', fontsize=12)
#     plt.ylabel('Feature 2', fontsize=12)
#     plt.legend(fontsize=10)
#     plt.grid(True, alpha=0.3)

#     plt.tight_layout()
#     plt.show()

#     # Print numerical results
#     print(f"\nOptimal Centroids (Silhouette Score: {-best_fit:.4f}):")
#     print(best_centroids)

#     return best_centroids, -best_fit

In [ ]:
# # Now you can call the function and get the results:
# best_centroids, best_score = runn_genetic_algorithm(X)

# *EVALUATING MODELS*

In [ ]:
# print(X_scaled.shape)  # Should show (n_samples, n_features)
# print(labels.shape)    # Should show (n_samples,)

In [ ]:
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

# After getting your final clusters
labels = np.argmin(cdist(X, best_centroids), axis=1)

# Silhouette Score (-1 to 1, higher is better)
sil_score = silhouette_score(X, labels)
print(f"Silhouette Score: {sil_score:.4f}")

# Davies-Bouldin Index (0 to ∞, lower is better)
db_index = davies_bouldin_score(X, labels)
print(f"Davies-Bouldin Index: {db_index:.4f}")

# Calinski-Harabasz Index (higher is better)
ch_index = calinski_harabasz_score(X, labels)
print(f"Calinski-Harabasz Index: {ch_index:.4f}")

In [ ]:
# # Run multiple times and compare results
# n_runs = 2
# results = []
# for _ in range(n_runs):
#     centroids, score = run_genetic_algorithm(X)
#     labels = np.argmin(cdist(X, centroids), axis=1)
#     results.append(labels)

# # Compare using Adjusted Rand Index
# from sklearn.metrics import adjusted_rand_score
# for i in range(n_runs-1):
#     print(f"Consistency between run {i} and {i+1}:",
#           adjusted_rand_score(results[i], results[i+1]))

In [ ]:
# # Cluster sizes
# cluster_sizes = np.bincount(labels)
# print("Cluster sizes:", cluster_sizes)

# # Intra-cluster distances
# intra_dists = [np.mean(pairwise_distances(X[labels==i])) 
#                for i in range(NUM_CLUSTERS)]
# print("Mean intra-cluster distances:", intra_dists)

# # Inter-cluster distances
# inter_dists = pairwise_distances(best_centroids)
# print("Inter-centroid distances:\n", inter_dists)

In [ ]:
# # Compare feature distributions across clusters
# fig, axes = plt.subplots(NUM_FEATURES, 1, figsize=(10, NUM_FEATURES*3))
# for i in range(NUM_FEATURES):
#     for cluster in range(NUM_CLUSTERS):
#         axes[i].hist(X[labels==cluster, i], alpha=0.5, 
#                     label=f'Cluster {cluster}')
#     axes[i].set_title(f'Feature {i} Distribution')
#     axes[i].legend()
# plt.tight_layout()
# plt.show()

In [ ]:
from sklearn.cluster import KMeans, DBSCAN

# Compare with KMeans
kmeans = KMeans(n_clusters=NUM_CLUSTERS).fit(X)
print("KMeans Silhouette:", silhouette_score(X, kmeans.labels_))

# Compare with DBSCAN (if appropriate for your data)
dbscan = DBSCAN(eps=0.5, min_samples=5).fit(X)
if len(np.unique(dbscan.labels_)) > 1:
    print("DBSCAN Silhouette:", silhouette_score(X, dbscan.labels_))

In [ ]:
from sklearn.metrics import silhouette_score

def evaluate_silhouette(X, labels):
    score = silhouette_score(X, labels)
    print(f"Silhouette Score: {score:.4f}")
    return score


In [ ]:
from sklearn.cluster import KMeans

# Assuming X is your dataset and NUM_CLUSTERS is defined
kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=42)
kmeans.fit(X)

# Print Silhouette Score
evaluate_silhouette(X, kmeans.labels_)


In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

def evaluate_silhouette(X, labels):
    score = silhouette_score(X, labels)
    print(f"Silhouette Score: {score:.4f}")
    return score



# Fit KMeans to the data
kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=42)
kmeans.fit(X)

# Print Silhouette Score
evaluate_silhouette(X, kmeans.labels_)
